<a href="https://colab.research.google.com/github/AvivBGU/DeepLearning_Assignment2/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Constants

<font size="4">Imports </font>

In [2]:
import os
import torch.nn as nn
import zipfile

<font size="4">Constants</font>

In [3]:
current_working_directory = os.getcwd()
DATA_BASE_DIRECTORY: str = os.path.join(current_working_directory, 'data')

# Acquiring & Handling Data

<font size="6">Acquiring_Data</font>

In [4]:
!pip install -q gdown

# 1. Download from Google Drive using gdown (extract the file ID from your link)
file_id = "1p1wjaqpTh_5RHfJu4vUh8JJCdKwYMHCp"
zip_path = "dataset.zip"
!gdown {file_id} -O {zip_path}

# 2. Extract the zip
os.makedirs(DATA_BASE_DIRECTORY, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATA_BASE_DIRECTORY)

# 3. List the contents
!ls {DATA_BASE_DIRECTORY}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1p1wjaqpTh_5RHfJu4vUh8JJCdKwYMHCp
From (redirected): https://drive.google.com/uc?id=1p1wjaqpTh_5RHfJu4vUh8JJCdKwYMHCp&confirm=t&uuid=51df2ee5-c057-4f58-8ef0-de4a384498ac
To: /content/dataset.zip
100% 104M/104M [00:01<00:00, 59.3MB/s] 


<font size="6">Handling_Data</font>

In [ ]:
# Iterating over all the data.
# Dividing to images with 2 (or more) examples and others that have exactly one.
# tuple[dict_more_than_2_examples, dict_1_example]
def get_all_pictures_divided_by_examples(base_directory: str) -> tuple[dict[str, list[str]], dict[str, list[str]]]:
    returned_tuple: tuple[dict[str, list[str]], dict[str, list[str]]] = (dict(), dict())
    for root, subdirs, files in os.walk(base_directory):
        if root == base_directory:
            continue
        picture_name: str = root.split(os.sep)[-1]
        print(picture_name)
        list_to_insert: list[str] = list()
        for file in files:
            if not file.endswith('.jpg'):
                raise Warning(f"File {file} is not a .jpg file. Continuing...")
            list_to_insert.append(os.path.join(root, file))
        if len(list_to_insert) > 1:
            returned_tuple[0][picture_name] = list_to_insert
        elif len(list_to_insert) == 1:
            returned_tuple[1][picture_name] = list_to_insert
        else:
            # Length 0
            raise Warning(f"Found no pictures in: {root}")
    return returned_tuple

divided_images = get_all_pictures_divided_by_examples(DATA_BASE_DIRECTORY)

<font size="6">Creating Network</font>

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=10),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=7),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
        )

        self.fc1 = nn.Sequential(
            nn.Flatten(),                         # -> [256*6*6 = 9216]
            nn.Linear(256*6*6, 4096),
            nn.Sigmoid()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 1),
            nn.Sigmoid()  # similarity score
        )

    def forward_once(self, x):
        x = self.cnn(x)
        x = self.fc1(x)
        return x

    def forward(self, input1, input2):
        out1 = self.forward_once(input1)
        out2 = self.forward_once(input2)
        # L1 distance
        diff = torch.abs(out1 - out2)
        similarity_score = self.fc2(diff)
        return similarity_score



# asd = ConvNet()

NameError: name 'nn' is not defined